# 1: Importing the required libraries

we going to need keras , numpy , sklearn, pandas , nltk



In [ ]:

# Importing the required libraries
import keras
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense,Bidirectional
from nltk.tokenize import word_tokenize,sent_tokenize
from keras.layers import *
from sklearn.model_selection import cross_val_score
import nltk
import pandas as pd
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#2- Loading our dataset
 isear - International Survey on Emotion Antecedents and Reactions


 and chaning the names of the columns


In [ ]:
df=pd.read_csv('/content/isear.csv',header=None)
# Rename columns and reorder them
df.columns = ['emotion', 'text']
df.head()

,emotion,text
0,joy,[ On days when I feel close to my partner and ...
1,fear,Every time I imagine that someone I love or I ...
2,anger,When I had been obviously unjustly treated and...
3,sadness,When I think about the short time that we live...
4,disgust,At a gathering I found myself involuntarily si...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7575 entries, 0 to 7651
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  7575 non-null   object
 1   text     7575 non-null   object
dtypes: object(2)
memory usage: 435.6+ KB


drop the null data "No Response." from text column

In [ ]:
noResponse = df[df['text'] == '[ No response.]'].index
df.drop(noResponse, inplace=True)

#3:  Tokeization

Apply a word tokenizer to convert each sentence into a list of words.

ex : if  sentence- ‘I am sad. after tokenizing it will get converted into a list [‘I’,’am’, sad].

In [ ]:
#select the text column
emotion_arr=df['text']

#first row
emotion_arr[0]

'[ On days when I feel close to my partner and other friends.  \nWhen I feel at peace with myself and also experience a close \ncontact with people whom I regard greatly.]'

In [ ]:
# loop over the text using word_tokenize
emotion_arr=[word_tokenize(sent) for sent in emotion_arr]
print(emotion_arr[0])

['[', 'On', 'days', 'when', 'I', 'feel', 'close', 'to', 'my', 'partner', 'and', 'other', 'friends', '.', 'When', 'I', 'feel', 'at', 'peace', 'with', 'myself', 'and', 'also', 'experience', 'a', 'close', 'contact', 'with', 'people', 'whom', 'I', 'regard', 'greatly', '.', ']']


 # 4: Padding
Padding in NLP (Natural Language Processing) is like adding extra spaces or symbols to make all sentences in a dataset have the same length.

In [ ]:
# Defined a function padd in which each sentence length is fixed to 100.
# If length is less than 100 , then the word- '<padd>' is append
def padd(arr):
    for i in range(100-len(arr)):
        arr.append('<pad>')
    return arr[:100]

# call the padd function for each sentence in emotion_arr
for i in range(len(emotion_arr)):
  emotion_arr[i]=padd(emotion_arr[i])

In [ ]:
print(emotion_arr[0])

['[', 'On', 'days', 'when', 'I', 'feel', 'close', 'to', 'my', 'partner', 'and', 'other', 'friends', '.', 'When', 'I', 'feel', 'at', 'peace', 'with', 'myself', 'and', 'also', 'experience', 'a', 'close', 'contact', 'with', 'people', 'whom', 'I', 'regard', 'greatly', '.', ']', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


Connect to my Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#5:Word embedding using the glove

Each word needs a numeric representation for the model to understand it, as the model only works with numbers. To do this, we got a ready-made set of 50-number vectors called "GloVe vectors" from the internet. These vectors help convert words into these 50-number representations.

The GloVe vector collection covers nearly all English words.

In each row, the first word is the one we're converting into numbers. The remaining columns in that row correspond to the 50 numbers in the vector.

In [ ]:
vocab_f ='/content/glove.6B.50d.txt'

In [ ]:


# embeddings_index is a dictionary which contains the mapping of
# word with its corresponding 50d vector.
embeddings_index = {}
with open(vocab_f, encoding='utf8') as f:
    for line in f:
        # splitting each line of the glove.6B.50d in a list of
        # items- in which the first element is the word to be embedded,
        # and from second to the end of line contains the 50d vector.
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# the embedding index of word 'happy'
embeddings_index['happy']

array([ 0.092086,  0.2571  , -0.58693 , -0.37029 ,  1.0828  , -0.55466 ,
       -0.78142 ,  0.58696 , -0.58714 ,  0.46318 , -0.11267 ,  0.2606  ,
       -0.26928 , -0.072466,  1.247   ,  0.30571 ,  0.56731 ,  0.30509 ,
       -0.050312, -0.64443 , -0.54513 ,  0.86429 ,  0.20914 ,  0.56334 ,
        1.1228  , -1.0516  , -0.78105 ,  0.29656 ,  0.7261  , -0.61392 ,
        2.4225  ,  1.0142  , -0.17753 ,  0.4147  , -0.12966 , -0.47064 ,
        0.3807  ,  0.16309 , -0.323   , -0.77899 , -0.42473 , -0.30826 ,
       -0.42242 ,  0.055069,  0.38267 ,  0.037415, -0.4302  , -0.39442 ,
        0.10511 ,  0.87286 ], dtype=float32)

In [ ]:

# Embedding each word of the emotion_arr
embedded_emotion_arr = []

for each_sentence in emotion_arr:
    embedded_emotion_arr.append([])
    for word in each_sentence:
        if word.lower() in embeddings_index:
            embedded_emotion_arr[-1].append(embeddings_index[word.lower()])
        else:
            # if the word to be embedded is '<padd>' append 0 fifty times
            embedded_emotion_arr[-1].append([0]*50)

print(embedded_emotion_arr[0][0])

[-0.61201   0.98226   0.11539   0.014623  0.23873  -0.067035  0.30632
 -0.64742  -0.38517  -0.03691   0.094788  0.57631  -0.091557 -0.54825
  0.25255  -0.14759   0.13023   0.21658  -0.30623   0.30028  -0.23471
 -0.17927   0.9518    0.54258   0.31172  -0.51038  -0.65223  -0.48858
  0.13486  -0.40132   2.493    -0.38777  -0.26456  -0.49414  -0.3871
 -0.20983   0.82941  -0.46253   0.39549   0.014881  0.79485  -0.79958
 -0.16243   0.013862 -0.53536   0.52536   0.019818 -0.16353   0.30649
  0.81745 ]


#6: One Hot encoding for the target variables and split train and test dataset

Do one-hot encoding of each emotion.

Split the dataset into train and test sets.

In [ ]:
# Perform one-hot encoding from sklearn
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
Y = enc.fit_transform(np.array(df['emotion']).reshape(-1,1)).toarray()
X=np.array(embedded_emotion_arr)


In [ ]:



# Split into train and test
from keras.layers import Embedding
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#create model



  ```
  def model(X, Y, input_size1, input_size2, output_size)
  ```
  :: This is the function definition that takes input data (X), target labels (Y), input size dimensions (input_size1 and input_size2), and output size (output_size) as parameters.
  ```
  m = Sequential()
  ```
  :Creates a Sequential model, which is a linear stack of layers.
  ```
  m.add(Bidirectional(LSTM(100, input_shape=(input_size1, input_size2))))
  ```

  Adds a Bidirectional LSTM layer with 100 units to the model. The input_shape parameter specifies the shape of the input data.
  ```
  m.add(Dropout(0.5))
  ```
  Adds a dropout layer that randomly drops 50% of the connections to prevent overfitting.

  ```
  m.add(Dense(output_size, activation='softmax'))
  ```
  Adds a fully connected Dense layer with a size of output_size units and a softmax activation function, which is often used for multi-class classification problems.

  ```
  m.compile('Adam', 'categorical_crossentropy', ['accuracy'])
  ```
  Compiles the model with the Adam optimizer, categorical cross-entropy loss (suitable for multiclass classification), and accuracy metric.

  ```
  m.fit(X, Y, epochs=32, batch_size=128)
  ```
  Trains the model on the input data X and target labels Y for 32 epochs using batches of size 128.
  ```
  return m
  ```
  Returns the compiled and trained model.

In [ ]:
# Defining the BiLSTM Model
def model(X, Y, input_size1, input_size2, output_size):
    m = Sequential()  # Create a Sequential model

    # Add a Bidirectional LSTM layer with 100 units
    m.add(Bidirectional(LSTM(100, input_shape=(input_size1, input_size2))))

    m.add(Dropout(0.5))  # Add a dropout layer with 50% dropout rate

    # Add a Dense (fully connected) layer with 'output_size' units and softmax activation
    m.add(Dense(output_size, activation='softmax'))

    # Compile the model with 'Adam' optimizer, 'categorical_crossentropy' loss, and 'accuracy' metric
    m.compile('Adam', 'categorical_crossentropy', ['accuracy'])

    # Train the model on input data 'X' and target labels 'Y' for 32 epochs, using batches of size 128
    m.fit(X, Y, epochs=32, batch_size=128)

    return m  # Return the compiled and trained model


In [ ]:
bilstmModel=model(X_train,Y_train,100,50,7)

Epoch 1/32
48/48 [==============================] - 35s 605ms/step - loss: 1.9187 - accuracy: 0.1957
Epoch 2/32
48/48 [==============================] - 24s 500ms/step - loss: 1.7789 - accuracy: 0.3119
Epoch 3/32
48/48 [==============================] - 25s 508ms/step - loss: 1.6928 - accuracy: 0.3523
Epoch 4/32
48/48 [==============================] - 25s 524ms/step - loss: 1.6610 - accuracy: 0.3658
Epoch 5/32
48/48 [==============================] - 22s 469ms/step - loss: 1.6105 - accuracy: 0.3850
Epoch 6/32
48/48 [==============================] - 26s 551ms/step - loss: 1.5916 - accuracy: 0.3919
Epoch 7/32
48/48 [==============================] - 24s 499ms/step - loss: 1.5655 - accuracy: 0.4035
Epoch 8/32
48/48 [==============================] - 23s 478ms/step - loss: 1.5344 - accuracy: 0.4271
Epoch 9/32
48/48 [==============================] - 25s 515ms/step - loss: 1.4978 - accuracy: 0.4429
Epoch 10/32
48/48 [==============================] - 22s 464ms/step - loss: 1.4677 - accura

# 7: save the model


In [ ]:
bilstmModel.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_1 (Bidirectio  (None, 200)              120800    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 7)                 1407      
                                                                 
Total params: 122,207
Trainable params: 122,207
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Testing the model
bilstmModel.evaluate(X_test,Y_test)

48/48 [==============================] - 3s 40ms/step - loss: 1.5878 - accuracy: 0.4528


[1.587786078453064, 0.4528052806854248]

#8 : predict using our model

In [ ]:
# Preprocess input text for prediction
def preprocess_input_text(text):
    tokenized_text = word_tokenize(text)
    padded_text = padd(tokenized_text)

    embedded_text = []
    for word in padded_text:
        if word.lower() in embeddings_index:
            embedded_text.append(embeddings_index[word.lower()])
        else:
            embedded_text.append([0]*50)
    return np.array(embedded_text)


In [ ]:
input_text = "I am feeling happy today."
processed_input = preprocess_input_text(input_text)


In [ ]:
# Perform prediction using the loaded model
predicted_probs = bilstmModel.predict(np.array([processed_input]))
predicted_emotion_index = np.argmax(predicted_probs)

predicted_emotion_index

1/1 [==============================] - 0s 38ms/step


4

In [ ]:
# Decode the predicted emotion
predicted_emotion = enc.categories_[0][predicted_emotion_index]

In [ ]:
print("Predicted Emotion:", predicted_emotion)
print("Predicted Emotion Probabilities:", predicted_probs)

Predicted Emotion: joy
Predicted Emotion Probabilities: [[0.00853515 0.01681765 0.00696166 0.00258888 0.7698855  0.16200666
  0.03320459]]


LSTM

Imagine you're reading a story, but you're reading it both forwards and backwards at the same time. That's kind of like what a Bidirectional LSTM does, but with words instead of a story.

LSTM stands for Long Short-Term Memory, which is a type of neural network that's really good at understanding sequences of data, like sentences. It's used a lot in things like language translation and text generation.

Now, a Bidirectional LSTM takes this idea a step further. Instead of just looking at words in one direction (like reading a sentence from start to finish), it looks at words both forwards and backwards. This helps it capture more context from the surrounding words.

Imagine you're trying to understand the meaning of a word in a sentence. Sometimes the words that come after it give you a clue, and sometimes the words before it do. A Bidirectional LSTM makes sure you're not missing out on any of those clues.

